# PSET 3: Exploring Taylor Expansions and DDP

...

Try not to add additional cells nor rearrange any that exist now as it will mess with our autograder (but feel free to open another Colab/notebook on the side).

### Make sure to submit your final notebook with all of your solutions to Gradescope!
[Direct Gradescope Link](https://www.gradescope.com/courses/820552)

### Before Starting

This problem set is written in only Python, but we do have a few "hacks" in place to display graphics in the cell outputs of this interactive notebook.  We'll also need a few libraries:

In [ ]:
!pip3 install "pygame>2.1" "numpy>1.23" "scipy>1.9"

## Part 1: Taylor Approximations of Dynamics and Cost Functions (7 Points)

In this problem set, we'll first implement a few dynamics and cost helper functions that we'll use in the larger trajopt functions.  Functions to update:
* `next_state`
* `next_state_gradient`
* `cost_value`
* `cost_gradient`
* `cost_hessian`

**Note:** most things are of type [`np.array`](https://numpy.org/doc/stable/user/quickstart.html) which hopefully should simplify your linear algebra and there are a few more hints left in the code comments! Also, make sure to take into account both the states and the controls!

---

### Step 1: `next_state`
We'll begin by converting the pendulum physics functions which compute acceleration (`dynamics`) into `next_state`.  We assume a pendulum with the following variables:

- $q$: Position (angle) of the pendulum.
- $\dot{q}$ (or `qd`): Velocity (angular velocity) of the pendulum.
- $\ddot{q}$ (or `qdd`): Acceleration (angular acceleration) of the pendulum.

The pendulum dynamics—i.e., acceleration—are influenced by:
- Gravity ($g$), which depends on the pendulum's position.
- $\text{damping}$, coefficient which depends on the pendulum's velocity.
- A control input $u$, which directly affects the acceleration.

$$
\ddot{q} = u - g \cdot \sin(q) - \text{damping} \cdot \dot{q}
$$

The dynamics are computed for you as follows:

```python
def dynamics(self, x, u):
   position = x[0]
   velocity = x[1]
   qdd = u - self.gravity*sin(position) - self.damping*velocity
   return np.array(qdd)
```

Recall from class that Euler integration is used to approximate the next state of the system by stepping forward in time with a small time step $dt$ (`self.timestep`). The update equations for the state variables $q$ (position) and $\dot{q}$ (velocity) are calculated by adding a small amount of acceleration and velocity to the original position and velocity.

$$
q' = q + dt \cdot \dot{q}
$$
$$
\dot{q}' = \dot{q} + dt \cdot \ddot{q}
$$

Where:
- $q'$ (or `q'`) is the next position.
- $\dot{q}'$ (or `qd'` in the code) is the next velocity.
- $\ddot{q}$ (or `qdd`) is the acceleration, computed via the provided `dynamics()` function.

So, for `next_state`, you should return a vector
```text
[q', qd'] = [q, qd] + dt * [qd, qdd]
```

---

### Step 2: `next_state_gradient`

The dynamics of the system depend on both the current state $[q, \dot{q}]$ and the control input $u$. We define two gradient matrices:

```text
             A = [[dq'/dq, dq'/dqd],    and B = [[dq/du],
                  [dqd'/dq, dqd'/dqd]]           [dqd/du]]
```

- $\mathbf{A}$: The gradient of the next state with respect to the current state $[q, \dot{q}]$.
- $\mathbf{B}$: The gradient of the next state with respect to the control input $u$.

In mathematical notation (if it helps):
$$
A = \begin{bmatrix} 
        \frac{\partial q'}{\partial q} & \frac{\partial q'}{\partial \dot{q}} \\[10pt]
        \frac{\partial \dot{q}'}{\partial q} & \frac{\partial \dot{q}'}{\partial \dot{q}} 
    \end{bmatrix}
$$

$$
B = \begin{bmatrix} 
        \frac{\partial q}{\partial u} \\[10pt]
        \frac{\partial \dot{q}}{\partial u} 
    \end{bmatrix}
$$


Your task is to construct these matrices which includes taking partial derivatives of $q'$ (`q'`) and $\dot{q}'$ (`qd'`) which you computed in Step 1 (but their equations were provided above, anyway) and also applying the chain rule.  (Yes, this was done in examples in lecture slides where $h$ represents the timestep `dt` or `self.timestep`.  But try to do this yourself with the notation provided in this problem set!)

We (hopefully) have made your life easier by providing the partial derivatives of $\ddot{q}$ (i.e., acceleration or `qdd`) to show how small changes in the current state and control input affect the acceleration:

1. Derivative of acceleration with respect to position: $\frac{\partial \ddot{q}}{\partial q} = -g \cdot \cos(q)$
2. Derivative of acceleration with respect to velocity: $\frac{\partial \ddot{q}}{\partial \dot{q}} = -\text{damping}$

3. Derivative of acceleration with respect to the control input: $\frac{\partial \ddot{q}}{\partial u} = 1$

```python
def dynamics_gradient(self, x, u):
    position = x[0]  # q
    velocity = x[1]  # qd
    # acceleration (or qdd) = u - self.gravity*sin(position) - self.damping*velocity

    dqdd_dq = -self.gravity * np.cos(position)
    dqdd_dqd = -self.damping
    dqdd_du = 1
    return np.array([dqdd_dq, dqdd_dqd, dqdd_du])
```

---

### Step 3: Cost Helper Functions

Next, we'll move onto the cost functions. Here we need to implement the cost value, gradient, and Hessian. For this problem, we are going to use the "standard simple quadratic cost":

$$
\text{cost}(x,u) = 0.5(x-x_g)^T Q (x-x_g) + 0.5 u^T R u
$$

- $x$: State variable vector (i.e., $\begin{bmatrix} q, \dot{q}\end{bmatrix}$).
- $u$: Control input
- $x_g$: Goal state.
- $\mathbf{Q}$: A matrix that weighs the state cost.
- $\mathbf{R}$: A matrix that weighs the control cost.

As that outputs a scalar value, the cost gradient and Hessian, therefore, are of the form:

```text
g = [dcost/dq, dcost/dqd, dcost/du]
H = [[d^2cost/dq^2,   d^2cost/dq dqd, d^2cost/dq du],
     [d^2cost/dqd dq, dcost/dqd^2,    d^2cost/dqd du],
     [d^2cost/du dq,  dcost/du dqd,   d^2cost/du^2]]
```

In mathematical notation:
$$\mathbf{g} = \begin{bmatrix}\frac{\partial \text{cost}}{\partial x}, \frac{\partial \text{cost}}{\partial u}\end{bmatrix}$$

$$\mathbf{H} = \begin{bmatrix}
\frac{\partial^2 \text{cost}}{\partial q^2} & \frac{\partial^2 \text{cost}}{\partial q \, \partial \dot{q}} & \frac{\partial^2 \text{cost}}{\partial q \, \partial u} \\[10pt]
\frac{\partial^2 \text{cost}}{\partial \dot{q} \, \partial q} & \frac{\partial^2 \text{cost}}{\partial \dot{q}^2} & \frac{\partial^2 \text{cost}}{\partial \dot{q} \, \partial u} \\[10pt]
\frac{\partial^2 \text{cost}}{\partial u \, \partial q} & \frac{\partial^2 \text{cost}}{\partial u \, \partial \dot{q}} & \frac{\partial^2 \text{cost}}{\partial u^2}
\end{bmatrix}$$



In this step, please implement the following functions:

1. `cost_value(x, u)` - computes the cost value based on the formula provided.
2. `cost_gradient(x, u)` - computes the gradient of the cost value with respect to the states and control inputs.
3. `cost_hessian(x, u)` - computes the Hessian of the cost value.

$Q$ and $R$ are provided (find them in `self.Q` and `self.R`), as is the goal $x_g$ (in `self.goal`). And you may find our `block_diag()` helper useful in this step (e.g., when constructing the Hessian).

In [ ]:
from math import sin, cos, pi
import numpy as np
import copy

class Pendulum():
    def __init__(self, timestep = 0.15, gravity = 9.81, damping = 0.01, control_min = -5, control_max = 5, control_step = 0.25):
        self.timestep = timestep
        self.gravity = gravity
        self.damping = damping
        self.control_min = control_min
        self.control_max = control_max
        self.control_step = control_step
        self.Q = None
        self.R = None
        self.QF = None
        self.goal = None

    ######################
    #                    #
    # High level helpers #
    #                    #
    ######################

    # return the state and control size
    def get_state_size(self):
        return 2
    def get_control_size(self):
        return 1

    # clamp the input n to the range [smallest, largest]
    def clamp(self, n, smallest, largest):
        return max(smallest, min(n, largest))

    # ensure angles are between -pi and pi
    def wrap_angles(self, angle):
        upper_bound = pi
        lower_bound = -pi
        while (angle < lower_bound):
            angle += 2*pi
        while (angle > upper_bound):
            angle -= 2*pi
        return np.array(angle)

    # compute the difference between two states making sure to wrap angles
    def state_delta(self, state1, state2):
        return np.array([self.wrap_angles(state1[0]-state2[0]), state1[1]-state2[1]])

    #####################################################
    #                                                   #
    #  Helper functions you need to fill out start here #
    #                                                   #
    #####################################################

    ######################
    #                    #
    #  Physics helpers   #
    #                    #
    ######################

    # apply the physics of the pendulum to compute acceleration
    def dynamics(self, x, u):
        position = x[0]
        velocity = x[1]
        # acceleration (or qdd) = u - self.gravity*sin(position) - self.damping*velocity
        qdd = u - self.gravity*sin(position) - self.damping*velocity
        return np.array(qdd)

    # compute the gradient of applying physics of the pendulum to compute acceleration
    def dynamics_gradient(self, x, u):
        position = x[0]
        velocity = x[1]

        # acceleration (or qdd) = u - self.gravity*sin(position) - self.damping*velocity
        dqdd_dq = -self.gravity*cos(position)   # derivative of qdd w.r.t. q
        dqdd_dqd = -self.damping                # derivative of qdd w.r.t. qd
        dqdd_du = 1                             # derivative of qdd w.r.t. u
        return np.array([dqdd_dq, dqdd_dqd, dqdd_du])

    # compute the next state using euler integration
    def next_state(self, x, u):
        position = np.array(x[0])
        velocity = np.array(x[1])

        # first compute acceleration using dynamics
        # note: in this case we also clamp the input to control_min/max
        #       to avoid unrealistic steps (this can be safely ignored in the gradient)
        u_clamp = self.clamp(u, self.control_min, self.control_max)
        acceleration = self.dynamics(x,u_clamp)

        #
        # TODO
        #
        # Euler integrator [q', qd'] = [q, qd] + dt * [qd, qdd]
        # note: * make sure to wrap_angles where appropriate
        #       * position and velocity are already numpy arrays
        #         so keep that in mind when constructing the next state
        #         return value
        #
        ...

        return np.array([0,0])


    # compute the gradient of the next state function using euler integration
    def next_state_gradient(self, x, u):
        position = x[0]
        velocity = x[1]
        A = np.array([[0, 0], [0, 0]])
        B = np.array([[0], [0]])

        #
        # TODO
        #
        # Euler integrator [q', qd'] = [q, qd] + dt * [qd, qdd]
        # Return the partial derivative matricies:
        #     A = [[dq'/dq, dq'/dqd],    and B = [[dq/du],
        #          [dqd'/dq, dqd'/dqd]]           [dqd/du]]
        #
        #
        ...
        return A, B



    ######################
    #                    #
    #    Cost helpers    #
    #                    #
    ######################

    def set_Q(self, Q):
        self.Q = Q
    def set_R(self, R):
        self.R = R
    def set_QF(self, QF):
        self.QF = QF
    def set_goal(self, goal):
        self.goal = goal

    # compute the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    def cost_value(self, x, u = None):
        cost = 0
        # first compute the error between the current state and the goal
        delta = self.state_delta(x, self.goal)
        # then compute the cost for that error
        #
        # TODO
        #
        # hint: np.matmul may be useful!
        ...
        return 0

    # compute the gradient of the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    #
    # return the vector [dcost/dq, dcost/dqd, dcost/du]
    def cost_gradient(self, x, u = None):
        grad = []
        # first compute the error between the current state and the goal
        delta = self.state_delta(x, self.goal)
        # then compute the cost gradient with respect to the state
        #
        # TODO
        #
        if u is not None:
            # hint: np.matmul and np.hstack may be useful!
            ...
            return np.array([0,0,0])
        else:
            ...
            return np.array([0,0])

    # compute the hessian of the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    #
    # return the matrix [[d^2cost/dq^2,   d^2cost/dq dqd, d^2cost/dq du],
    #                    [d^2cost/dqd dq, dcost/dqd^2,    d^2cost/dqd du],
    #                    [d^2cost/du dq,  dcost/du dqd,   d^2cost/du^2]]
    #
    # Hint: you may find the block_diag helper function helpful! (But you don't need to use it)
    #
    def cost_hessian(self, x, u = None):
        H = [[]]
        #
        # TODO
        #
        # hint: if u is None then you only need the hessian with respect to q, qd
        #
        if u is not None:
            ...
            return np.zeros((3,3))
        else:
            ...
            return np.zeros((2,2))

    def cost_gradient_hessian(self, x, u = None):
        return self.cost_hessian(x,u), self.cost_gradient(x,u)

    def block_diag(self, *arrs):
        """Create a block diagonal matrix from the provided arrays.

        Given the inputs `A`, `B` and `C`, the output will have these
        arrays arranged on the diagonal::

            [[A, 0, 0],
             [0, B, 0],
             [0, 0, C]]

        If all the input arrays are square, the output is known as a
        block diagonal matrix.

        Parameters
        ----------
        A, B, C, ... : array-like, up to 2D
            Input arrays.  A 1D array or array-like sequence with length n is
            treated as a 2D array with shape (1,n).

        Returns
        -------
        D : ndarray
            Array with `A`, `B`, `C`, ... on the diagonal.  `D` has the
            same dtype as `A`.

        References
        ----------
        .. [1] Wikipedia, "Block matrix",
               http://en.wikipedia.org/wiki/Block_diagonal_matrix

        Examples
        --------
        >>> A = [[1, 0],
        ...      [0, 1]]
        >>> B = [[3, 4, 5],
        ...      [6, 7, 8]]
        >>> C = [[7]]
        >>> print(block_diag(A, B, C))
        [[1 0 0 0 0 0]
         [0 1 0 0 0 0]
         [0 0 3 4 5 0]
         [0 0 6 7 8 0]
         [0 0 0 0 0 7]]
        >>> block_diag(1.0, [2, 3], [[4, 5], [6, 7]])
        array([[ 1.,  0.,  0.,  0.,  0.],
               [ 0.,  2.,  3.,  0.,  0.],
               [ 0.,  0.,  0.,  4.,  5.],
               [ 0.,  0.,  0.,  6.,  7.]])

        """
        if arrs == ():
            arrs = ([],)
        arrs = [np.atleast_2d(a) for a in arrs]

        bad_args = [k for k in range(len(arrs)) if arrs[k].ndim > 2]
        if bad_args:
            raise ValueError("arguments in the following positions have dimension "
                                "greater than 2: %s" % bad_args)

        shapes = np.array([a.shape for a in arrs])
        out = np.zeros(np.sum(shapes, axis=0), dtype=arrs[0].dtype)

        r, c = 0, 0
        for i, (rr, cc) in enumerate(shapes):
            out[r:r + rr, c:c + cc] = arrs[i]
            r += rr
            c += cc
        return out

## Part 2 - Explore
Now that you have a working helper functions we'll use the DDP implementation below to allow us to solve some trajectory optimization problems.

In [ ]:
# This program runs the main algorithms built on top of your util functions
# (Do not modify, but please read)

import math, pygame, copy, time
import IPython.display as ipydisplay
from pygame.locals import *
from time import sleep
import numpy as np
import scipy as sp
np.set_printoptions(precision=3)

class Trajopt:
    def __init__(self, robot_object, start_node, goal_node, N, XMAX_MIN, YMAX_MIN, \
                       MAX_ITER = 100, EXIT_TOL = 1e-3, ALPHA_FACTOR = 0.5, ALPHA_MIN = 1e-4, MU = 5):
        self.robot_object = robot_object # the robot_object with physics and cost functions
        self.MAX_ITER = MAX_ITER         # total ddp loops to try
        self.EXIT_TOL = EXIT_TOL         # This is the convergence criterion. We will declare success when the trajectory
                                         # is updated by a norm of less that 1e-4. DO NOT MODIFY.
        self.N = N                       # Number of nodes in a trajectory
        self.start_node = start_node
        self.goal_node = goal_node
        self.XMAX_MIN = XMAX_MIN         # max for drawing locically
        self.YMAX_MIN = YMAX_MIN         # max for drawing locically
        self.canvas_max = 640            # max for drawing pixels
        # set global line search parameters
        self.alpha_factor = ALPHA_FACTOR # how much to reduce alpha by each deeper search
        self.alpha_min = ALPHA_MIN       # minimum alpha to try
        self.mu = MU                     # merit function weighting

    def draw_circle(self, node, size, color):
        percent_x = (node[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y = (node[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN

        scaled_x = int(self.canvas_max*percent_x)
        scaled_y = int(self.canvas_max*percent_y)

        pygame.draw.circle(self.screen, color, (scaled_x, scaled_y), size)

    def draw_line(self, node1, node2, color):
        percent_x1 = (node1[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y1 = (node1[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN
        percent_x2 = (node2[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y2 = (node2[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN

        scaled_x1 = int(self.canvas_max*percent_x1)
        scaled_y1 = int(self.canvas_max*percent_y1)
        scaled_x2 = int(self.canvas_max*percent_x2)
        scaled_y2 = int(self.canvas_max*percent_y2)

        # check if angle wrapping (and don't draw lines across the screen)
        if node1[0] - node2[0] > math.pi:
            pass
        elif node1[0] - node2[0] < -math.pi:
            pass
        else:
            pygame.draw.line(self.screen, color, (scaled_x2, scaled_y2), (scaled_x1, scaled_y1))

    def init_screen(self):
        # initialize and prepare screen
        pygame.init()
        self.screen = pygame.display.set_mode((self.canvas_max,self.canvas_max))
        pygame.display.set_caption('PS3 - DDP')
        black = 20, 20, 40
        blue = 0, 0, 255
        green = 0, 255, 0
        self.screen.fill(black)
        self.draw_circle(self.start_node, 5, blue)
        self.draw_circle(self.goal_node, 5, green)
        pygame.display.update()

    def draw_trajectory(self, x):
        white = 255, 240, 200
        red = 255, 0, 0

        # Draw the trajectory
        self.draw_circle(x[:,0], 2, white)
        for k in range(1, self.N):
            self.draw_circle(x[:,k], 2, white)
            self.draw_line(x[:,k-1], x[:,k], red)

        pygame.display.update()

        # Capture the screen and display in Colab
        pygame.image.save(self.screen, "/tmp/screenshot.png")
        ipydisplay.display(ipydisplay.Image("/tmp/screenshot.png"))

        time.sleep(0.4)

    def solve(self, x, u, N, DISPLAY_MODE = False):
        # start up the graphics
        self.init_screen()

        # get constants
        nx = self.robot_object.get_state_size()
        nu = self.robot_object.get_control_size()

        # solve
        self.iLQR(x, u, nx, nu, N, DISPLAY_MODE)


    #####################################################
    #                                                   #
    #                    iLQR Helpers                   #
    #                                                   #
    #####################################################

    # Compute the total cost along the trajectory
    def compute_total_cost(self, x, u, nx, nu, N):
        J = 0
        for k in range(N-1):
            J += self.robot_object.cost_value(x[:,k], u[:,k])
        J += self.robot_object.cost_value(x[:,N-1])
        return J

    # compute the dynamics and cost gradient and hessians
    def compute_approximation(self, xk, uk, nx, nu, k):
        Gk, gk = self.robot_object.cost_gradient_hessian(xk, uk)
        Ak, Bk = self.robot_object.next_state_gradient(xk, uk)
        return Ak, Bk, Gk, gk

    # put most of the above backward pass functions together and get from
    # the inputs to return the current kappa, K, Vxx, Vx
    def backward_pass_iterate(self, A, B, H, g, Vxx_kp1, Vx_kp1, nx, nu, k):
        # Backpropogate the CTG through the dynamics
        Hxx = np.matmul(A.transpose(),np.matmul(Vxx_kp1,A)) + H[0:nx,0:nx]
        Huu = np.matmul(B.transpose(),np.matmul(Vxx_kp1,B)) + H[nx:,nx:]
        Hux = np.matmul(B.transpose(),np.matmul(Vxx_kp1,A)) + H[nx:,0:nx]
        gx = np.matmul(A.transpose(),Vx_kp1) + g[0:nx]
        gu = np.matmul(B.transpose(),Vx_kp1) + g[nx:]

        # Invert Huu block
        HuuInv = np.linalg.inv(Huu)

        # Compute feedback and feedforward updates
        K = -np.matmul(HuuInv,Hux)
        kappa = -np.matmul(HuuInv,gu)

        # Compute the next CTG estimate
        Vxx = Hxx - np.matmul(Hux.transpose(),K)
        Vx = gx - np.matmul(Hux.transpose(),kappa)

        return kappa, K, Vxx, Vx

    # return u_new based on the current x_new, k, kappa, alpha, and original x
    def compute_control_update(self, x, x_new, K, kappa, alpha, nx, nu, N):
        delta = self.robot_object.state_delta(x_new, x)
        return alpha*kappa + np.matmul(K,delta)

    # rollout the full trajectory to produce x_new, u_new based on
    # x, u (the original trajectory), as well as K, kappa, alpha (the updates)
    def rollout_trajectory(self, x, u, K, kappa, alpha, nx, nu, N):
        x_new = copy.deepcopy(x)
        u_new = copy.deepcopy(u)
        for k in range(N-1):
            u_new[:,k] = u[:,k] + self.compute_control_update(x[:,k], x_new[:,k], K[:,:,k], kappa[:,k], alpha, nx, nu, k)
            x_new[:,k+1] = self.robot_object.next_state(x_new[:,k], u_new[:,k])
        return x_new, u_new

    # main iLQR function
    def iLQR(self, x, u, nx, nu, N, DISPLAY_MODE = False):

        # allocate memory for things we will compute
        kappa = np.zeros([nu,N-1])     # control updates
        K = np.zeros((nu,nx,N-1))   # feedback gains

        # compute initial cost
        J = self.compute_total_cost(x, u, nx, nu, N)
        J_prev = J
        if DISPLAY_MODE:
            print("Initial Cost: ", J)

        # start the main loop
        iteration = 0
        failed = False
        while 1:

            # Do backwards pass to compute new control update and feedback gains: kappa and K
            # start by initializing the cost to go
            Vxx, Vx = self.robot_object.cost_gradient_hessian(x[:,N-1])
            for k in range(N-2,-1,-1):
                # then compute the quadratic approximation at that point
                A, B, H, g = self.compute_approximation(x[:,k], u[:,k], nx, nu, k)

                # then compute the control update and new CTG estimates
                kappak, Kk, Vxx, Vx = self.backward_pass_iterate(A, B, H, g, Vxx, Vx, nx, nu, k)

                # save kappa and K for forward pass
                kappa[:,k] = kappak.tolist()
                K[:,:,k] = Kk.tolist()

            # Do forwards pass to compute new x, u, J (with line search)
            alpha = 1
            while 1:
                # rollout new trajectory
                x_new, u_new = self.rollout_trajectory(x, u, K, kappa, alpha, nx, nu, N)

                # compute new cost
                J_new = self.compute_total_cost(x_new, u_new, nx, nu, N)

                # simple line search criteria
                if J_new < J:
                    J_prev = J
                    x = x_new
                    u = u_new
                    J = J_new
                    if DISPLAY_MODE:
                        print("Iteration[", iteration, "] with cost[", round(J,4), "] and x final:")
                        print(x[:,N-1])
                        self.draw_trajectory(x)
                    break

                # failed try to continue the line search
                elif alpha > self.alpha_min:
                    alpha *= self.alpha_factor
                    if DISPLAY_MODE:
                        print("Deepening the line search")

                # failed line search
                else:
                    if DISPLAY_MODE:
                        print("Line search failed")
                    failed = True
                    break

            # Check for exit (or error)
            if failed: # need double break to get out of both loops here if line search fails
                break

            delta_J = J_prev - J
            if delta_J < self.EXIT_TOL:
                if DISPLAY_MODE:
                    print("Exiting for exit_tolerance")
                break

            if iteration == self.MAX_ITER - 1:
                if DISPLAY_MODE:
                    print("Exiting for max_iter")
                break
            else:
                iteration += 1

        if DISPLAY_MODE:
            print("Final Trajectory")
            print(x)
            print(u)

Let's change a few environment variables to [trick](https://colab.research.google.com/drive/1xtiBrGeRHmXY3KSOixkZBf_rJIgBImJu?usp=sharing#scrollTo=7X42jJWlAuSl) Google Colab into running pygame without error.

In [ ]:
import os
# set SDL to use the dummy NULL video driver,
#   so it doesn't need a windowing system.
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:
from math import pi
import numpy as np

XMAX_MIN = pi
YMAX_MIN = 10

X_START = [0,0]
X_GOAL = [pi,0]

# How did I get these numbers? A little intuition and a lot of guess and check.
QF = np.array([[30,0],[0,10]])
Q = np.array([[3,0],[0,1]])
R = np.array([0.08])

N = 32

pend = Pendulum()
pend.set_Q(Q)
pend.set_R(R)
pend.set_QF(QF)
pend.set_goal(X_GOAL)

x0 = np.zeros([pend.get_state_size(),N])
u0 = np.zeros([pend.get_control_size(),N-1])

trajopt_obj = Trajopt(pend, X_START, X_GOAL, N, XMAX_MIN, YMAX_MIN)

## Run the solver!
Run the solver. What happens? How does this work as compared to the SQP algorithm? Is it better? Worse? Slower? Faster? Does the trajectory go all the way to the goal? If you change the parameters for the cost function by adjusting `Q` and `R` in that file what happens? Play around with it a little bit. Hopefully it will give you a little better intuition for what the math is doing!

In [ ]:
trajopt_obj.solve(x0, u0, N, True)